In [2]:
library(tidyverse)
library(haven)
library(ivreg)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


ERROR: Error in library(ivreg): there is no package called ‘ivreg’


In [4]:
census_data <- read_dta('cen_ind_2021_pumf_v2 2.dta')

In [14]:
statistic <- census_data |> select(agegrp, yrim, lfact, marsth, hdgree, jobperm,CFInc_AT, PR1, NOC21) |> 
    filter(agegrp != 88) |> 
    filter(agegrp != 88) |>
    filter(agegrp != 88) |>
    filter(agegrp != 88) |>
    filter(agegrp != 88) |>
    filter(agegrp != 88) |>
  summarise(
    across(everything(),
      list(
        mean = ~ mean(.x, na.rm = TRUE),
        sd   = ~ sd(.x, na.rm = TRUE),
        min  = ~ min(.x, na.rm = TRUE),
        max  = ~ max(.x, na.rm = TRUE)
      )
    )
  )
head(statistic)

agegrp_mean,agegrp_sd,agegrp_min,agegrp_max,yrim_mean,yrim_sd,yrim_min,yrim_max,lfact_mean,lfact_sd,⋯,CFInc_AT_min,CFInc_AT_max,PR1_mean,PR1_sd,PR1_min,PR1_max,NOC21_mean,NOC21_sd,NOC21_min,NOC21_max
<dbl>,<dbl>,<dbl+lbl>,<dbl+lbl>,<dbl>,<dbl>,<dbl+lbl>,<dbl+lbl>,<dbl>,<dbl>,⋯,<dbl+lbl>,<dbl+lbl>,<dbl>,<dbl>,<dbl+lbl>,<dbl+lbl>,<dbl>,<dbl>,<dbl+lbl>,<dbl+lbl>
11.36212,5.217992,1,21,7992.141,3702.859,1,9999,21.24788,35.09513,⋯,1,88,37.7443,15.38871,10,99,51.44171,42.80216,1,99


In [34]:
census_subset <- census_data %>% filter(agegrp == 16 | agegrp == 17 ) |>
    mutate(retired = ifelse(lfact %in% c(11,12,13), 1, 0)) |>
    mutate(female = ifelse(Gender == 1, 1, 0))
head(census_subset)

ppsort,aboid,agegrp,ageimm,attsch,bfnmemb,bedrm,CFInc,CFInc_AT,cfstat,⋯,WT10,WT11,WT12,WT13,WT14,WT15,WT16,rand,retired,female
<dbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
897,6,16,99,1,0,3,26,25,2,⋯,22.89379,22.89379,22.89379,22.89379,22.89379,22.89379,22.89379,2.129959,1,1
977,6,17,1,1,0,5,11,10,7,⋯,22.89379,22.89379,22.89379,22.89379,249.19883,22.89379,22.89379,2.006523,1,0
1640,6,16,99,1,0,5,25,22,1,⋯,22.90134,22.90134,22.90134,22.90134,22.90134,22.90134,22.90134,2.060246,0,0
2362,6,17,99,1,0,2,9,9,7,⋯,22.89379,22.89379,22.89379,22.89379,22.89379,22.89379,22.89379,2.596228,1,1
2452,6,16,5,1,0,4,18,17,2,⋯,22.89379,22.89379,22.89379,22.89379,22.89379,22.89379,22.89379,2.890442,0,0
2539,6,16,99,1,0,2,22,19,6,⋯,22.89379,22.89379,22.89379,249.19883,22.89379,22.89379,22.89379,2.203408,0,0


In [16]:
census_subset <- census_subset |>
mutate(eligible = ifelse((agegrp == 17 & immstat == 1) | ((immstat %in% c(2, 3)) & (2021 - yrim) >= 10), 1, 0))
head(census_subset)

ERROR: Error: object 'census_subset' not found


In [42]:
model1 <- lm(retired ~ eligible, data = census_subset) 
summary(model1)


Call:
lm(formula = retired ~ eligible, data = census_subset)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.6492 -0.3891  0.3508  0.3508  0.6109 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.38912    0.01174   33.13   <2e-16 ***
eligible     0.26013    0.01721   15.12   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.4829 on 3163 degrees of freedom
Multiple R-squared:  0.06738,	Adjusted R-squared:  0.06709 
F-statistic: 228.5 on 1 and 3163 DF,  p-value: < 2.2e-16


In [46]:
model2 <- lm(retired ~ eligible + female + factor(marsth), data = census_subset) 
summary(model2)


Call:
lm(formula = retired ~ eligible + female + factor(marsth), data = census_subset)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.7782 -0.4464  0.2218  0.4289  0.7086 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)      0.34922    0.02959  11.803  < 2e-16 ***
eligible         0.25640    0.01713  14.968  < 2e-16 ***
female           0.13170    0.01717   7.671 2.26e-14 ***
factor(marsth)2 -0.03449    0.02977  -1.159    0.247    
factor(marsth)3 -0.02078    0.03878  -0.536    0.592    
factor(marsth)4  0.03096    0.05655   0.547    0.584    
factor(marsth)5 -0.05786    0.03737  -1.548    0.122    
factor(marsth)6  0.04083    0.04614   0.885    0.376    
factor(marsth)8  0.02258    0.24085   0.094    0.925    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.4782 on 3156 degrees of freedom
Multiple R-squared:  0.08749,	Adjusted R-squared:  0.08518 
F-statistic: 37.83 on 8 and 3156 DF,  p-value:

In [49]:
model3 <- lm(retired ~ eligible + female + factor(marsth) + factor(hdgree) + factor(jobperm), data = census_subset) 
summary(model3)


Call:
lm(formula = retired ~ eligible + female + factor(marsth) + factor(hdgree) + 
    factor(jobperm), data = census_subset)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.9482 -0.1454  0.0486  0.1493  0.9713 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)       0.028698   0.030830   0.931   0.3520    
eligible          0.097518   0.012979   7.513 7.47e-14 ***
female            0.032446   0.013016   2.493   0.0127 *  
factor(marsth)2   0.036613   0.021931   1.669   0.0951 .  
factor(marsth)3   0.035689   0.028501   1.252   0.2106    
factor(marsth)4   0.034313   0.041631   0.824   0.4099    
factor(marsth)5   0.016925   0.027448   0.617   0.5375    
factor(marsth)6   0.036355   0.033897   1.073   0.2836    
factor(marsth)8   0.043338   0.176603   0.245   0.8062    
factor(hdgree)2   0.031754   0.019161   1.657   0.0976 .  
factor(hdgree)3   0.021671   0.030722   0.705   0.4806    
factor(hdgree)4   0.080648   0.034911   2.310   0.0209 

In [54]:
model4 <- lm(retired ~ eligible + female + factor(marsth) + factor(hdgree) + factor(jobperm) + CFInc_AT	 + factor(PR1) + factor(NOC21), data = census_subset) 
summary(model4)


Call:
lm(formula = retired ~ eligible + female + factor(marsth) + factor(hdgree) + 
    factor(jobperm) + CFInc_AT + factor(PR1) + factor(NOC21), 
    data = census_subset)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.96916 -0.15909  0.03059  0.14422  0.99506 

Coefficients: (1 not defined because of singularities)
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)      -0.0830365  0.0881398  -0.942   0.3462    
eligible          0.0983987  0.0130428   7.544 5.94e-14 ***
female            0.0340382  0.0140600   2.421   0.0155 *  
factor(marsth)2   0.0570732  0.0235901   2.419   0.0156 *  
factor(marsth)3   0.0640263  0.0298580   2.144   0.0321 *  
factor(marsth)4   0.0370806  0.0421237   0.880   0.3788    
factor(marsth)5   0.0179733  0.0276238   0.651   0.5153    
factor(marsth)6   0.0404460  0.0342130   1.182   0.2372    
factor(marsth)8   0.0318543  0.1769469   0.180   0.8571    
factor(hdgree)2   0.0367935  0.0194551   1.891   0.0587 .  
factor

In [66]:
census_subset <- census_subset %>% mutate(yob = 2021 - agegrp)

In [67]:
iv_model <- ivreg(retired ~ eligible + female + factor(marsth) + factor(hdgree) + factor(jobperm) + CFInc_AT + factor(PR1) + factor(NOC21) | yob + female + factor(marsth) + factor(hdgree) + factor(jobperm) + CFInc_AT + factor(PR1) + factor(NOC21), data = census_subset)

In [69]:
iv_model


Call:
ivreg(formula = retired ~ eligible + female + factor(marsth) +     factor(hdgree) + factor(jobperm) + CFInc_AT + factor(PR1) +     factor(NOC21) | yob + female + factor(marsth) + factor(hdgree) +     factor(jobperm) + CFInc_AT + factor(PR1) + factor(NOC21),     data = census_subset)

Coefficients:
     (Intercept)          eligible            female   factor(marsth)2  
       -0.083147          0.098832          0.034041          0.057025  
 factor(marsth)3   factor(marsth)4   factor(marsth)5   factor(marsth)6  
        0.064001          0.037091          0.017913          0.040350  
 factor(marsth)8   factor(hdgree)2   factor(hdgree)3   factor(hdgree)4  
        0.031796          0.036791          0.037951          0.082389  
 factor(hdgree)5   factor(hdgree)6   factor(hdgree)7   factor(hdgree)8  
        0.043700          0.043897          0.091479          0.035561  
 factor(hdgree)9  factor(hdgree)10  factor(hdgree)11  factor(hdgree)12  
        0.053684          0.042684   